---

### **1. Import Library**

In [30]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
from linearmodels.panel import PanelOLS, RandomEffects
from statsmodels.stats.diagnostic import het_breuschpagan
from scipy.stats import chi2
from scipy.stats import f


---

### **2. Load Data**

In [31]:
data = pd.read_excel("data_panel_ekonomi_indonesia.xlsx")
data


,Tahun,Provinsi,Tingkat Pengangguran Terbuka (Persen),Indeks Pembangunan Manusia,Produk Domestik Regional Bruto per Kapita HB (Rp)
0,2019,ACEH,6.17,71.90,164162.98
1,2020,ACEH,6.59,71.99,166372.32
2,2021,ACEH,6.30,72.18,184979.88
3,2022,ACEH,6.17,72.80,210418.36
4,2023,ACEH,6.03,74.70,227110.20
...,...,...,...,...,...
165,2019,PAPUA,3.51,60.84,189510.70
166,2020,PAPUA,4.28,60.44,199186.57
167,2021,PAPUA,3.33,60.62,235487.47
168,2022,PAPUA,2.83,61.39,262519.79


---

### **3. Preprocessing Data**
Konversi data ke dalam format panel dengan indeks multi-level:

In [32]:
data['Tahun'] = pd.to_datetime(data['Tahun'], format='%Y')
data.set_index(['Provinsi', 'Tahun'], inplace=True)
data = data.sort_index()

# Cek data setelah preprocessing
print(data.info())

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 170 entries, ('ACEH', Timestamp('2019-01-01 00:00:00')) to ('SUMATERA UTARA', Timestamp('2023-01-01 00:00:00'))
Data columns (total 3 columns):
 #   Column                                             Non-Null Count  Dtype  
---  ------                                             --------------  -----  
 0   Tingkat Pengangguran Terbuka (Persen)              170 non-null    float64
 1   Indeks Pembangunan Manusia                         170 non-null    float64
 2   Produk Domestik Regional Bruto per Kapita HB (Rp)  170 non-null    float64
dtypes: float64(3)
memory usage: 5.9+ KB
None


---

### **4. Definisikan Variabel**
Pisahkan variabel dependen (`y`) dan independen (`X`):

In [33]:
# Variabel dependen (Y)
y = data['Tingkat Pengangguran Terbuka (Persen)']

# Variabel independen (X)
X = data[['Indeks Pembangunan Manusia', 'Produk Domestik Regional Bruto per Kapita HB (Rp)']]
X = sm.add_constant(X)  # Tambahkan konstanta


---

### **5. Uji Heteroskedastisitas**
Uji Breusch-Pagan untuk memeriksa heteroskedastisitas:
- **Heteroskedastisitas**: Jika p-value < 0.05, ada indikasi heteroskedastisitas.


In [34]:
ols_model = sm.OLS(y, X).fit()

# Breusch-Pagan Test
bp_test = het_breuschpagan(ols_model.resid, ols_model.model.exog)
print("Hasil Uji Heteroskedastisitas (Breusch-Pagan):")
print(f"Lagrange Multiplier: {bp_test[0]}, p-value: {bp_test[1]}")


Hasil Uji Heteroskedastisitas (Breusch-Pagan):
Lagrange Multiplier: 5.645814623351758, p-value: 0.05943290132026017


---

### **6. Uji Chow**
Uji Chow mengidentifikasi apakah ada perubahan struktural dalam data, misalnya sebelum dan sesudah tahun tertentu:
- **Uji Chow**: Jika p-value < 0.05, ada perubahan struktural antara dua subkelompok.


In [ ]:
# Membagi data menjadi dua periode (Sebelum covid-19 dan setelah covid-19)
data_2019_2021 = data.loc[data.index.get_level_values('Tahun') <= '2020']
data_2022_2023 = data.loc[data.index.get_level_values('Tahun') > '2020']

# Menghitung RSS untuk masing-masing model
ols_1 = sm.OLS(data_2019_2021['Tingkat Pengangguran Terbuka (Persen)'],
               sm.add_constant(data_2019_2021[['Indeks Pembangunan Manusia', 
                                               'Produk Domestik Regional Bruto per Kapita HB (Rp)']])).fit()
ols_2 = sm.OLS(data_2022_2023['Tingkat Pengangguran Terbuka (Persen)'],
               sm.add_constant(data_2022_2023[['Indeks Pembangunan Manusia', 
                                               'Produk Domestik Regional Bruto per Kapita HB (Rp)']])).fit()

RSS_pooled = ols_model.ssr
RSS_1 = ols_1.ssr
RSS_2 = ols_2.ssr

n1, n2 = len(data_2019_2021), len(data_2022_2023)
k = X.shape[1]

# Hitung statistik F untuk uji Chow
F_chow = ((RSS_pooled - (RSS_1 + RSS_2)) / k) / ((RSS_1 + RSS_2) / (n1 + n2 - 2 * k))
p_value_chow = 1 - f.cdf(F_chow, dfn=k, dfd=(n1 + n2 - 2 * k))

print("Hasil Uji Chow:")
print(f"F-statistik = {F_chow}, p-value = {p_value_chow}")


Hasil Uji Chow:
F-statistik = 1.7891763821849598, p-value = 0.15125939031087143


---

### **7. Model Data Panel**
Hitung model **Fixed Effects** dan **Random Effects**:

In [13]:
# Fixed Effects
fixed_model = PanelOLS(y, X, entity_effects=True).fit()
print("Hasil Fixed Effects:")
print(fixed_model)

# Random Effects
random_model = RandomEffects(y, X).fit()
print("Hasil Random Effects:")
print(random_model)


Hasil Fixed Effects:
                                    PanelOLS Estimation Summary                                    
Dep. Variable:     Tingkat Pengangguran Terbuka (Persen)   R-squared:                        0.1589
Estimator:                                      PanelOLS   R-squared (Between):             -3.3680
No. Observations:                                    170   R-squared (Within):               0.1589
Date:                                   Sat, Nov 30 2024   R-squared (Overall):             -2.7833
Time:                                           00:37:41   Log-likelihood                   -169.11
Cov. Estimator:                               Unadjusted                                           
                                                           F-statistic:                      12.653
Entities:                                             34   P-value                           0.0000
Avg Obs:                                          5.0000   Distribution:       

---

### **8. Uji Hausman**
Gunakan uji Hausman untuk membandingkan Fixed Effects dan Random Effects:
- **Fixed vs Random**: 
  - Jika uji Hausman signifikan (p-value < 0.05), gunakan Fixed Effects.
  - Jika tidak signifikan, gunakan Random Effects.


In [18]:
b_fixed = fixed_model.params.values
b_random = random_model.params.values
cov_diff = random_model.cov.values - fixed_model.cov.values

# Menghitung statistik Hausman
hausman_stat = ((b_fixed - b_random).T @ np.linalg.inv(cov_diff) @ (b_fixed - b_random))

# Menghitung p-value menggunakan distribusi chi-square
p_value_hausman = 1 - chi2.cdf(hausman_stat, df=len(b_fixed))

print("Hasil Uji Hausman:")
print(f"Statistik: {hausman_stat}, p-value: {p_value_hausman}")


Hasil Uji Hausman:
Statistik: -42.89574329785529, p-value: 1.0


---

### **9. Uji Autokorelasi (Durbin-Watson Test)**

Uji Durbin-Watson digunakan untuk mendeteksi adanya autokorelasi dalam residual model regresi. Nilai Durbin-Watson berkisar dari 0 hingga 4:
- Nilai mendekati **2**: Tidak ada autokorelasi.
- Nilai mendekati **0**: Autokorelasi positif.
- Nilai mendekati **4**: Autokorelasi negatif.

---

### **Interpretasi Hasil Durbin-Watson**
- **Nilai mendekati 2**: Residual tidak memiliki autokorelasi (baik untuk model).
- **Nilai jauh dari 2**: Ada kemungkinan autokorelasi dalam residual.
  - Jika nilai mendekati **0**, ada autokorelasi positif.
  - Jika nilai mendekati **4**, ada autokorelasi negatif.

In [23]:
# Uji Autokorelasi Durbin-Watson untuk Fixed Effects
dw_fixed = sm.stats.durbin_watson(fixed_model.resids)
print("Hasil Uji Durbin-Watson untuk Fixed Effects:")
print(f"Durbin-Watson Statistic: {dw_fixed}")

# Uji Autokorelasi Durbin-Watson untuk Random Effects
dw_random = sm.stats.durbin_watson(random_model.resids)
print("Hasil Uji Durbin-Watson untuk Random Effects:")
print(f"Durbin-Watson Statistic: {dw_random}")


Hasil Uji Durbin-Watson untuk Fixed Effects:
Durbin-Watson Statistic: 2.135614115687824
Hasil Uji Durbin-Watson untuk Random Effects:
Durbin-Watson Statistic: 1.4649176367531265
